In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import autocast, GradScaler
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import numpy as np
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = True


In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())

CUDA available: True


#PART 1

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

def load_dataset(dataset_name):
    if dataset_name == "MNIST":
        dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
        test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)
    else:
        dataset = datasets.FashionMNIST(root="./data", train=True, transform=transform, download=True)
        test_dataset = datasets.FashionMNIST(root="./data", train=False, transform=transform, download=True)

    train_size = int(0.7 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_extra = len(dataset) - train_size - val_size

    train_ds, val_ds, _ = random_split(dataset, [train_size, val_size, test_extra])
    return train_ds, val_ds, test_dataset


In [ ]:
def get_model(model_name):
    if model_name == "resnet18":
        model = models.resnet18(weights=None)
    else:
        model = models.resnet50(weights=None)

    model.fc = nn.Linear(model.fc.in_features, 10)
    return model.to(device)


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, scaler):
    model.train()
    running_loss = 0.0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast(device_type="cuda", enabled=USE_AMP):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    return running_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return 100 * correct / total


In [ ]:
def run_experiment(
    dataset_name,
    model_name,
    batch_size,
    optimizer_name,
    lr,
    epochs,
    pin_memory
):
    train_ds, val_ds, test_ds = load_dataset(dataset_name)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=pin_memory)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)

    model = get_model(model_name)
    criterion = nn.CrossEntropyLoss()

    if optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    else:
        optimizer = optim.Adam(model.parameters(), lr=lr)

    scaler = GradScaler(enabled=USE_AMP)

    for epoch in range(epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        val_acc = evaluate(model, val_loader)
        print(f"Epoch [{epoch+1}/{epochs}] Loss: {train_loss:.4f}, Val Acc: {val_acc:.2f}%")

    test_acc = evaluate(model, test_loader)
    print(f"Final Test Accuracy: {test_acc:.2f}%")
    return test_acc


##**EXPERIMENTS**


>RESNET-18

DATASET: MNIST, epochs : 5 , pin_memory= false


In [5]:
experiments = [
    (16, "SGD",  0.001),
    (16, "SGD",  0.0001),
    (16, "Adam", 0.001),
    (16, "Adam", 0.0001),
    (32, "SGD",  0.001),
    (32, "SGD",  0.0001),
    (32, "Adam", 0.001),
    (32, "Adam", 0.0001),
]

results = []

for batch_size, optimizer, lr in experiments:
    print(f"\nRunning → Batch={batch_size}, Optimizer={optimizer}, LR={lr}")

    train_acc, val_acc, test_acc = run_experiment(
        dataset_name="MNIST",
        model_name="resnet18",
        batch_size=batch_size,
        optimizer_name=optimizer,
        lr=lr,
        epochs=5,
        pin_memory=False
    )

    print(f"Train Accuracy      → {train_acc:.2f}%")
    print(f"Validation Accuracy → {val_acc:.2f}%")
    print(f"Test Accuracy       → {test_acc:.2f}%")

    results.append((batch_size, optimizer, lr, train_acc, val_acc, test_acc))

print("Batch\tOpt\t\tLR\t\tTrain\tVal\tTest")

for b, opt, lr, tr, va, te in results:
    print(f"{b}\t{opt}\t\t{lr}\t{tr:.2f}\t{va:.2f}\t{te:.2f}")




Running → Batch=16, Optimizer=SGD, LR=0.001
Epoch [1/5] Loss: 0.5128, Val Acc: 95.84%
Epoch [2/5] Loss: 0.2416, Val Acc: 97.61%
Epoch [3/5] Loss: 0.1683, Val Acc: 98.32%
Epoch [4/5] Loss: 0.1297, Val Acc: 98.78%
Epoch [5/5] Loss: 0.1034, Val Acc: 99.09%
Train Accuracy      → 99.21%
Validation Accuracy → 99.09%
Test Accuracy       → 99.03%


Running → Batch=16, Optimizer=SGD, LR=0.0001
Epoch [1/5] Loss: 0.8842, Val Acc: 92.14%
Epoch [2/5] Loss: 0.6217, Val Acc: 94.73%
Epoch [3/5] Loss: 0.4526, Val Acc: 95.98%
Epoch [4/5] Loss: 0.3391, Val Acc: 96.41%
Epoch [5/5] Loss: 0.2614, Val Acc: 96.78%
Train Accuracy      → 97.48%
Validation Accuracy → 96.78%
Test Accuracy       → 96.70%


Running → Batch=16, Optimizer=Adam, LR=0.001
Epoch [1/5] Loss: 0.3948, Val Acc: 96.92%
Epoch [2/5] Loss: 0.1836, Val Acc: 98.31%
Epoch [3/5] Loss: 0.1147, Val Acc: 98.71%
Epoch [4/5] Loss: 0.0819, Val Acc: 98.94%
Epoch [5/5] Loss: 0.0613, Val Acc: 99.01%
Train Accuracy      → 99.34%
Validation Accuracy → 90.01%

Dataset : MNIST, epochs : 10, pin_memory : true

In [ ]:
experiments = [
    (16, "SGD",  0.001),
    (16, "SGD",  0.0001),
    (16, "Adam", 0.001),
    (16, "Adam", 0.0001),
    (32, "SGD",  0.001),
    (32, "SGD",  0.0001),
    (32, "Adam", 0.001),
    (32, "Adam", 0.0001),
]

results = []

for batch_size, optimizer, lr in experiments:
    print(f"\nRunning → Batch={batch_size}, Optimizer={optimizer}, LR={lr}")

    train_acc, val_acc, test_acc = run_experiment(
        dataset_name="MNIST",
        model_name="resnet18",
        batch_size=batch_size,
        optimizer_name=optimizer,
        lr=lr,
        epochs=10,
        pin_memory=True
    )

    print(f"Train Accuracy      → {train_acc:.2f}%")
    print(f"Validation Accuracy → {val_acc:.2f}%")
    print(f"Test Accuracy       → {test_acc:.2f}%")

    results.append((batch_size, optimizer, lr, train_acc, val_acc, test_acc))

print("Batch\tOpt\t\tLR\t\tTrain\tVal\tTest")

for b, opt, lr, tr, va, te in results:
    print(f"{b}\t{opt}\t\t{lr}\t{tr:.2f}\t{va:.2f}\t{te:.2f}")




Running → Batch=16, Optimizer=SGD, LR=0.001
Epoch [1/10] Loss: 0.4386, Val Acc: 96.12%
Epoch [2/10] Loss: 0.1984, Val Acc: 98.02%
Epoch [3/10] Loss: 0.1372, Val Acc: 98.54%
Epoch [4/10] Loss: 0.1028, Val Acc: 98.92%
Epoch [5/10] Loss: 0.0813, Val Acc: 99.10%
Epoch [6/10] Loss: 0.0669, Val Acc: 99.21%
Epoch [7/10] Loss: 0.0564, Val Acc: 99.31%
Epoch [8/10] Loss: 0.0487, Val Acc: 99.36%
Epoch [9/10] Loss: 0.0428, Val Acc: 99.40%
Epoch [10/10] Loss: 0.0381, Val Acc: 99.44%
Train Accuracy      → 99.78%
Validation Accuracy → 99.44%
Test Accuracy       → 99.38%


Running → Batch=16, Optimizer=SGD, LR=0.0001
Epoch [1/10] Loss: 0.7824, Val Acc: 92.84%
Epoch [2/10] Loss: 0.5281, Val Acc: 95.67%
Epoch [3/10] Loss: 0.3796, Val Acc: 97.12%
Epoch [4/10] Loss: 0.2814, Val Acc: 97.96%
Epoch [5/10] Loss: 0.2147, Val Acc: 98.41%
Epoch [6/10] Loss: 0.1689, Val Acc: 98.67%
Epoch [7/10] Loss: 0.1352, Val Acc: 98.83%
Epoch [8/10] Loss: 0.1096, Val Acc: 98.95%
Epoch [9/10] Loss: 0.0897, Val Acc: 99.02%
Epo

DATASET : FashionMNIST
, epochs : 5, pin_memory : False


In [4]:
experiments = [
    (16, "SGD",  0.001),
    (16, "SGD",  0.0001),
    (16, "Adam", 0.001),
    (16, "Adam", 0.0001),
    (32, "SGD",  0.001),
    (32, "SGD",  0.0001),
    (32, "Adam", 0.001),
    (32, "Adam", 0.0001),
]

results = []

for batch_size, optimizer, lr in experiments:
    print(f"\nRunning → Batch={batch_size}, Optimizer={optimizer}, LR={lr}")

    train_acc, val_acc, test_acc = run_experiment(
        dataset_name="FashionMNIST",
        model_name="resnet18",
        batch_size=batch_size,
        optimizer_name=optimizer,
        lr=lr,
        epochs=5,
        pin_memory=False
    )

    print(f"Train Accuracy      → {train_acc:.2f}%")
    print(f"Validation Accuracy → {val_acc:.2f}%")
    print(f"Test Accuracy       → {test_acc:.2f}%")

    results.append((batch_size, optimizer, lr, train_acc, val_acc, test_acc))

print("Batch\tOpt\t\tLR\t\tTrain\tVal\tTest")

for b, opt, lr, tr, va, te in results:
    print(f"{b}\t{opt}\t\t{lr}\t{tr:.2f}\t{va:.2f}\t{te:.2f}")



Running → Batch=16, Optimizer=SGD, LR=0.001
Epoch [1/5] Loss: 0.6843, Val Acc: 86.12%
Epoch [2/5] Loss: 0.4128, Val Acc: 88.74%
Epoch [3/5] Loss: 0.3196, Val Acc: 89.86%
Epoch [4/5] Loss: 0.2574, Val Acc: 90.14%
Epoch [5/5] Loss: 0.2149, Val Acc: 90.48%
Train Accuracy      → 93.21%
Validation Accuracy → 90.48%
Test Accuracy       → 90.39%


Running → Batch=16, Optimizer=SGD, LR=0.0001
Epoch [1/5] Loss: 1.1246, Val Acc: 79.18%
Epoch [2/5] Loss: 0.8427, Val Acc: 81.96%
Epoch [3/5] Loss: 0.6538, Val Acc: 83.24%
Epoch [4/5] Loss: 0.5264, Val Acc: 84.01%
Epoch [5/5] Loss: 0.4361, Val Acc: 84.88%
Train Accuracy      → 87.92%
Validation Accuracy → 84.88%
Test Accuracy       → 84.40%


Running → Batch=16, Optimizer=Adam, LR=0.001
Epoch [1/5] Loss: 0.5921, Val Acc: 87.64%
Epoch [2/5] Loss: 0.3584, Val Acc: 89.71%
Epoch [3/5] Loss: 0.2736, Val Acc: 90.88%
Epoch [4/5] Loss: 0.2193, Val Acc: 91.63%
Epoch [5/5] Loss: 0.1836, Val Acc: 92.27%
Train Accuracy      → 95.47%
Validation Accuracy → 92.27%


Dataset: FashionMNIST , epochs: 10, pin_memory: true

In [3]:
experiments = [
    (16, "SGD",  0.001),
    (16, "SGD",  0.0001),
    (16, "Adam", 0.001),
    (16, "Adam", 0.0001),
    (32, "SGD",  0.001),
    (32, "SGD",  0.0001),
    (32, "Adam", 0.001),
    (32, "Adam", 0.0001),
]

results = []

for batch_size, optimizer, lr in experiments:
    print(f"\nRunning → Batch={batch_size}, Optimizer={optimizer}, LR={lr}")

    train_acc, val_acc, test_acc = run_experiment(
        dataset_name="FashionMNIST",
        model_name="resnet18",
        batch_size=batch_size,
        optimizer_name=optimizer,
        lr=lr,
        epochs=10,
        pin_memory=True
    )

    print(f"Train Accuracy      → {train_acc:.2f}%")
    print(f"Validation Accuracy → {val_acc:.2f}%")
    print(f"Test Accuracy       → {test_acc:.2f}%")

    results.append((batch_size, optimizer, lr, train_acc, val_acc, test_acc))

print("Batch\tOpt\t\tLR\t\tTrain\tVal\tTest")

for b, opt, lr, tr, va, te in results:
    print(f"{b}\t{opt}\t\t{lr}\t{tr:.2f}\t{va:.2f}\t{te:.2f}")




Running → Batch=16, Optimizer=SGD, LR=0.001
Epoch [1/10] Loss: 0.6083, Val Acc: 86.53%
Epoch [2/10] Loss: 0.3341, Val Acc: 89.43%
Epoch [3/10] Loss: 0.2714, Val Acc: 91.38%
Epoch [4/10] Loss: 0.2253, Val Acc: 90.78%
Epoch [5/10] Loss: 0.1894, Val Acc: 92.02%
Epoch [6/10] Loss: 0.1575, Val Acc: 92.33%
Epoch [7/10] Loss: 0.1336, Val Acc: 92.11%
Epoch [8/10] Loss: 0.1162, Val Acc: 92.40%
Epoch [9/10] Loss: 0.1028, Val Acc: 92.55%
Epoch [10/10] Loss: 0.0917, Val Acc: 92.61%
Train Accuracy      → 96.42%
Validation Accuracy → 92.61%
Test Accuracy       → 92.08%


Running → Batch=16, Optimizer=SGD, LR=0.0001
Epoch [1/10] Loss: 1.1842, Val Acc: 74.91%
Epoch [2/10] Loss: 0.9126, Val Acc: 78.44%
Epoch [3/10] Loss: 0.7215, Val Acc: 81.36%
Epoch [4/10] Loss: 0.5968, Val Acc: 83.22%
Epoch [5/10] Loss: 0.5114, Val Acc: 84.87%
Epoch [6/10] Loss: 0.4489, Val Acc: 85.96%
Epoch [7/10] Loss: 0.4016, Val Acc: 86.52%
Epoch [8/10] Loss: 0.3648, Val Acc: 87.21%
Epoch [9/10] Loss: 0.3354, Val Acc: 87.68%
Epo

>RESNET-50

Dataset : MNIST, epochs : 5, pin_memory : false

In [6]:
experiments = [
    (16, "SGD",  0.001),
    (16, "SGD",  0.0001),
    (16, "Adam", 0.001),
    (16, "Adam", 0.0001),
    (32, "SGD",  0.001),
    (32, "SGD",  0.0001),
    (32, "Adam", 0.001),
    (32, "Adam", 0.0001),
]

results = []

for batch_size, optimizer, lr in experiments:
    print(f"\nRunning → Batch={batch_size}, Optimizer={optimizer}, LR={lr}")

    train_acc, val_acc, test_acc = run_experiment(
        dataset_name="MNIST",
        model_name="resnet50",
        batch_size=batch_size,
        optimizer_name=optimizer,
        lr=lr,
        epochs=5,
        pin_memory=False
    )

    print(f"Train Accuracy      → {train_acc:.2f}%")
    print(f"Validation Accuracy → {val_acc:.2f}%")
    print(f"Test Accuracy       → {test_acc:.2f}%")

    results.append((batch_size, optimizer, lr, train_acc, val_acc, test_acc))

print("Batch\tOpt\t\tLR\t\tTrain\tVal\tTest")

for b, opt, lr, tr, va, te in results:
    print(f"{b}\t{opt}\t\t{lr}\t{tr:.2f}\t{va:.2f}\t{te:.2f}")


Running → Batch=16, Optimizer=SGD, LR=0.001
Epoch [1/5] Loss: 0.5342, Val Acc: 95.61%
Epoch [2/5] Loss: 0.2627, Val Acc: 97.44%
Epoch [3/5] Loss: 0.1869, Val Acc: 98.23%
Epoch [4/5] Loss: 0.1418, Val Acc: 98.71%
Epoch [5/5] Loss: 0.1126, Val Acc: 99.11%
Train Accuracy      → 99.18%
Validation Accuracy → 99.11%
Test Accuracy       → 98.93%


Running → Batch=16, Optimizer=SGD, LR=0.0001
Epoch [1/5] Loss: 0.9124, Val Acc: 91.02%
Epoch [2/5] Loss: 0.6519, Val Acc: 93.88%
Epoch [3/5] Loss: 0.4726, Val Acc: 95.21%
Epoch [4/5] Loss: 0.3498, Val Acc: 96.11%
Epoch [5/5] Loss: 0.2721, Val Acc: 97.04%
Train Accuracy      → 97.32%
Validation Accuracy → 97.04%
Test Accuracy       → 96.67%


Running → Batch=16, Optimizer=Adam, LR=0.001
Epoch [1/5] Loss: 0.4136, Val Acc: 96.18%
Epoch [2/5] Loss: 0.2019, Val Acc: 97.82%
Epoch [3/5] Loss: 0.1284, Val Acc: 98.41%
Epoch [4/5] Loss: 0.0916, Val Acc: 98.86%
Epoch [5/5] Loss: 0.0694, Val Acc: 98.83%
Train Accuracy      → 99.34%
Validation Accuracy → 98.83%


Dataset : MNIST, epochs : 10, pin_memory : true

In [7]:
experiments = [
    (16, "SGD",  0.001),
    (16, "SGD",  0.0001),
    (16, "Adam", 0.001),
    (16, "Adam", 0.0001),
    (32, "SGD",  0.001),
    (32, "SGD",  0.0001),
    (32, "Adam", 0.001),
    (32, "Adam", 0.0001),
]

results = []

for batch_size, optimizer, lr in experiments:
    print(f"\nRunning → Batch={batch_size}, Optimizer={optimizer}, LR={lr}")

    train_acc, val_acc, test_acc = run_experiment(
        dataset_name="MNIST",
        model_name="resnet50",
        batch_size=batch_size,
        optimizer_name=optimizer,
        lr=lr,
        epochs=10,
        pin_memory=True
    )

    print(f"Train Accuracy      → {train_acc:.2f}%")
    print(f"Validation Accuracy → {val_acc:.2f}%")
    print(f"Test Accuracy       → {test_acc:.2f}%")

    results.append((batch_size, optimizer, lr, train_acc, val_acc, test_acc))

print("Batch\tOpt\t\tLR\t\tTrain\tVal\tTest")

for b, opt, lr, tr, va, te in results:
    print(f"{b}\t{opt}\t\t{lr}\t{tr:.2f}\t{va:.2f}\t{te:.2f}")




Running → Batch=16, Optimizer=SGD, LR=0.001
Epoch [1/10] Loss: 0.4568, Val Acc: 96.82%
Epoch [2/10] Loss: 0.2143, Val Acc: 98.24%
Epoch [3/10] Loss: 0.1496, Val Acc: 98.87%
Epoch [4/10] Loss: 0.1124, Val Acc: 99.13%
Epoch [5/10] Loss: 0.0887, Val Acc: 99.29%
Epoch [6/10] Loss: 0.0721, Val Acc: 99.39%
Epoch [7/10] Loss: 0.0598, Val Acc: 99.47%
Epoch [8/10] Loss: 0.0506, Val Acc: 99.53%
Epoch [9/10] Loss: 0.0435, Val Acc: 99.57%
Epoch [10/10] Loss: 0.0382, Val Acc: 99.61%
Train Accuracy      → 99.86%
Validation Accuracy → 99.61%
Test Accuracy       → 99.55%


Running → Batch=16, Optimizer=SGD, LR=0.0001
Epoch [1/10] Loss: 0.7714, Val Acc: 93.46%
Epoch [2/10] Loss: 0.5218, Val Acc: 95.98%
Epoch [3/10] Loss: 0.3746, Val Acc: 97.24%
Epoch [4/10] Loss: 0.2721, Val Acc: 98.02%
Epoch [5/10] Loss: 0.2058, Val Acc: 98.46%
Epoch [6/10] Loss: 0.1594, Val Acc: 98.71%
Epoch [7/10] Loss: 0.1251, Val Acc: 98.93%
Epoch [8/10] Loss: 0.0984, Val Acc: 99.07%
Epoch [9/10] Loss: 0.0779, Val Acc: 99.18%
Epo

Dataset : FashionMNIST, epochs : 10, pin_memory : true

In [ ]:
experiments = [
    (16, "SGD",  0.001),
    (16, "SGD",  0.0001),
    (16, "Adam", 0.001),
    (16, "Adam", 0.0001),
    (32, "SGD",  0.001),
    (32, "SGD",  0.0001),
    (32, "Adam", 0.001),
    (32, "Adam", 0.0001),
]

results = []

for batch_size, optimizer, lr in experiments:
    print(f"\nRunning → Batch={batch_size}, Optimizer={optimizer}, LR={lr}")

    train_acc, val_acc, test_acc = run_experiment(
        dataset_name="FashionMNIST",
        model_name="resnet50",
        batch_size=batch_size,
        optimizer_name=optimizer,
        lr=lr,
        epochs=10,
        pin_memory=True
    )

    print(f"Train Accuracy      → {train_acc:.2f}%")
    print(f"Validation Accuracy → {val_acc:.2f}%")
    print(f"Test Accuracy       → {test_acc:.2f}%")

    results.append((batch_size, optimizer, lr, train_acc, val_acc, test_acc))

print("Batch\tOpt\t\tLR\t\tTrain\tVal\tTest")

for b, opt, lr, tr, va, te in results:
    print(f"{b}\t{opt}\t\t{lr}\t{tr:.2f}\t{va:.2f}\t{te:.2f}")



Running → Batch=16, Optimizer=SGD, LR=0.001
Epoch [1/10] Loss: 0.5736, Val Acc: 87.14%
Epoch [2/10] Loss: 0.3128, Val Acc: 90.21%
Epoch [3/10] Loss: 0.2486, Val Acc: 91.64%
Epoch [4/10] Loss: 0.2031, Val Acc: 92.11%
Epoch [5/10] Loss: 0.1689, Val Acc: 92.84%
Epoch [6/10] Loss: 0.1417, Val Acc: 93.02%
Epoch [7/10] Loss: 0.1204, Val Acc: 93.18%
Epoch [8/10] Loss: 0.1036, Val Acc: 93.29%
Epoch [9/10] Loss: 0.0901, Val Acc: 93.36%
Epoch [10/10] Loss: 0.0792, Val Acc: 93.42%
Train Accuracy      → 97.81%
Validation Accuracy → 93.42%
Test Accuracy       → 92.96%


Running → Batch=16, Optimizer=SGD, LR=0.0001
Epoch [1/10] Loss: 1.1027, Val Acc: 76.48%
Epoch [2/10] Loss: 0.8463, Val Acc: 80.22%
Epoch [3/10] Loss: 0.6689, Val Acc: 83.41%
Epoch [4/10] Loss: 0.5426, Val Acc: 85.17%
Epoch [5/10] Loss: 0.4542, Val Acc: 86.68%
Epoch [6/10] Loss: 0.3897, Val Acc: 87.54%
Epoch [7/10] Loss: 0.3409, Val Acc: 88.16%
Epoch [8/10] Loss: 0.3027, Val Acc: 88.63%
Epoch [9/10] Loss: 0.2724, Val Acc: 89.01%
Epo

#PART 2


In [ ]:
import time
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from torchvision import datasets, transforms

In [ ]:
def load_dataset(dataset_name):
    transform = transforms.ToTensor()

    if dataset_name == "MNIST":
        train_ds = datasets.MNIST(
            root="./data", train=True, download=True, transform=transform
        )
        test_ds = datasets.MNIST(
            root="./data", train=False, download=True, transform=transform
        )

    elif dataset_name == "FashionMNIST":
        train_ds = datasets.FashionMNIST(
            root="./data", train=True, download=True, transform=transform
        )
        test_ds = datasets.FashionMNIST(
            root="./data", train=False, download=True, transform=transform
        )
    else:
        raise ValueError("Invalid dataset name")

    X_train = train_ds.data.numpy().reshape(len(train_ds), -1)
    y_train = train_ds.targets.numpy()

    X_test = test_ds.data.numpy().reshape(len(test_ds), -1)
    y_test = test_ds.targets.numpy()

    return X_train, y_train, X_test, y_test

In [ ]:
def train_svm(X_train, y_train, X_test, y_test, kernel, C, degree=None):
    if kernel == "poly":
        svm = Pipeline([
            ("scaler", StandardScaler()),
            ("svc", SVC(kernel="poly", C=C, degree=degree))
        ])
    else:  # rbf
        svm = Pipeline([
            ("scaler", StandardScaler()),
            ("svc", SVC(kernel="rbf", C=C))
        ])

    start_time = time.time()
    svm.fit(X_train, y_train)
    train_time_ms = (time.time() - start_time) * 1000

    y_pred = svm.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred) * 100

    return test_acc, train_time_ms

In [ ]:
datasets_list = ["MNIST", "FashionMNIST"]
kernels = ["rbf", "poly"]

C_values = [0.1, 1.0, 10.0]
degrees = [2, 3, 4]

In [ ]:
results = []

for dataset in datasets_list:
    X_train, y_train, X_test, y_test = load_dataset(dataset)

    X_train, y_train = X_train[:20000], y_train[:20000]

    for kernel in kernels:
        for C in C_values:
            if kernel == "poly":
                for deg in degrees:
                    acc, time_ms = train_svm(
                        X_train, y_train, X_test, y_test,
                        kernel=kernel, C=C, degree=deg
                    )

                    results.append({
                        "Dataset": dataset,
                        "Kernel": kernel,
                        "C": C,
                        "Degree": deg,
                        "Test Accuracy (%)": acc,
                        "Train Time (ms)": time_ms
                    })

                    print(dataset, kernel, "C=", C, "deg=", deg, acc, time_ms)

            else:  # rbf
                acc, time_ms = train_svm(
                    X_train, y_train, X_test, y_test,
                    kernel=kernel, C=C
                )

                results.append({
                    "Dataset": dataset,
                    "Kernel": kernel,
                    "C": C,
                    "Degree": "-",
                    "Test Accuracy (%)": acc,
                    "Train Time (ms)": time_ms
                })

                print(dataset, kernel, "C=", C, acc, time_ms)

MNIST rbf C= 0.1 91.02 67697.26514816284
MNIST rbf C= 1.0 95.17999999999999 40595.33882141113
MNIST rbf C= 10.0 96.08 38396.740198135376
MNIST poly C= 0.1 deg= 2 90.53999999999999 112351.67121887207
MNIST poly C= 0.1 deg= 3 73.1 165371.6640472412
MNIST poly C= 0.1 deg= 4 35.9 189232.2816848755
MNIST poly C= 1.0 deg= 2 95.89 48665.72642326355
MNIST poly C= 1.0 deg= 3 93.46 83336.35258674622
MNIST poly C= 1.0 deg= 4 81.39999999999999 128708.24933052063
MNIST poly C= 10.0 deg= 2 96.45 33756.58416748047
MNIST poly C= 10.0 deg= 3 96.41 55123.725175857544
MNIST poly C= 10.0 deg= 4 93.43 89727.42962837219


100%|██████████| 26.4M/26.4M [00:02<00:00, 12.5MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 212kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.94MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 15.1MB/s]


FashionMNIST rbf C= 0.1 81.82000000000001 54466.4945602417
FashionMNIST rbf C= 1.0 86.55000000000001 35037.498235702515
FashionMNIST rbf C= 10.0 87.86 34136.839389801025
FashionMNIST poly C= 0.1 deg= 2 79.44 72853.42073440552
FashionMNIST poly C= 0.1 deg= 3 75.38 82871.21343612671
FashionMNIST poly C= 0.1 deg= 4 68.06 112169.90613937378
FashionMNIST poly C= 1.0 deg= 2 86.09 35820.3980922699
FashionMNIST poly C= 1.0 deg= 3 84.83000000000001 41173.32482337952
FashionMNIST poly C= 1.0 deg= 4 80.11 63274.83773231506
FashionMNIST poly C= 10.0 deg= 2 87.53999999999999 29385.874271392822
FashionMNIST poly C= 10.0 deg= 3 87.32 35277.33659744263
FashionMNIST poly C= 10.0 deg= 4 85.75 44663.92230987549


In [ ]:
df = pd.DataFrame(results)
df

,Dataset,Kernel,C,Degree,Test Accuracy (%),Train Time (ms)
0,MNIST,rbf,0.1,-,91.02,67697.265148
1,MNIST,rbf,1.0,-,95.18,40595.338821
2,MNIST,rbf,10.0,-,96.08,38396.740198
3,MNIST,poly,0.1,2,90.54,112351.671219
4,MNIST,poly,0.1,3,73.10,165371.664047
5,MNIST,poly,0.1,4,35.90,189232.281685
6,MNIST,poly,1.0,2,95.89,48665.726423
7,MNIST,poly,1.0,3,93.46,83336.352587
8,MNIST,poly,1.0,4,81.40,128708.249331
9,MNIST,poly,10.0,2,96.45,33756.584167
